# 特定のページで更新があった場合lineに通知するためのコード

> 更新頻度が不明な大学の奨学金のページで、新たな情報が記載されたかどうかを確認するのが面倒であった。そこで、情報の更新があれば通知をするというシステムを考えた。そして、そのコードが以下のコードである。また、コードの概要を示す。

## 概要

1.   特定のページからhtml形式の情報を取得する
2.   取得した情報を指定のディレクトリに保存する
3.   前日に取得したhtmlファイルと当日に取得したhtmlファイルを比較し一致している
     かどうかを確認する
4.   当日と前日取得した情報に差分があればlineで通知を行う




In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime
import os

data = requests.get("奨学金のサイト")


#soup=BeautifulSoup(res.content,"html.parser")

In [ ]:
path = "c:/line_notify"
with open('20230904_scholarship.html', 'w',encoding='utf-8') as file:
    file.write(data.text)

In [ ]:
#　pahtの指定
path = "c:/line_notify"
data = requests.get("奨学金のサイト")
with open('scholarship.html', 'w',encoding='utf-8') as file:
    file.write(data.text)

#取得したhtmlファイル名を変更_yyyymmdd_scholarship.html

now = datetime.datetime.now()
os.rename("scholarship.html", now.strftime("%Y%m%d") + "_scholarship.html")

#昨日のファイルを呼び出す

before = now - datetime.timedelta(days = 1 )
fp1='{0:%Y%m%d}_scholarship.html'.format(before)
filepath = str(path)+fp1

with open(filepath , encoding='utf-8') as f:
    html = f.read()
soup = BeautifulSoup(html, 'lxml')

#当日のファイルを呼び出す
fp2='{0:%Y%m%d}_scholarship.html'.format(now)
filepath2 = str(path)+fp2
with open(filepath2 , encoding='utf-8') as f:
    html = f.read()
    soup2 = BeautifulSoup(html, 'lxml')

#昨日のファイルと本日のファイルを比較して同じかどうかを確認

b = (soup == soup2)

#比較して違いがあるればlineAPIでメッセージを送信
def if_content(b):
    if b == True:
        print('ありません')
        os.remove(filepath)

    else:
        addmessage="奨学金のHPで" "\r\n" "情報の更新がありました。" "\r\n" "内容の確認をして下さい。" "\r\n" "奨学金のサイト"
        line_notify_token = '*************'
        line_notify_api = 'https://notify-api.line.me/api/notify'
        message = addmessage
        payload = {'message':"\r\n"+ message}
        headers = {'Authorization': 'Bearer ' + line_notify_token}  # 発行したトークン
        line_notify = requests.post(line_notify_api, data=payload, headers=headers)
        os.remove(filepath)
if_content(b)

